<a href="https://colab.research.google.com/github/natsakh/IAD/blob/main/Pr_7/7_3_Transformer_Decoder_IMDB_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

import matplotlib.pyplot as plt
import numpy as np
import re, string
from collections import Counter

import math
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import random

torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Device: cuda


In [2]:
# Hugging Face Datasets — бібліотека для завантаження датасетів.
#У Google Colab вона вже попередньо встановлена
# Якщо ви працюєте локально — попередньо виконайте:
#     pip install datasets
from datasets import load_dataset
#https://huggingface.co/datasets

In [22]:
ds = load_dataset("imdb")
#ds["train"], ds["test"]
#список словників — кожен елемент має ключі 'text' і 'label'

In [4]:
def clean_text(text):
    text = text.lower()                                       # до нижнього регістру
    text = re.sub(r"@\S+", " ", text)                         # прибрати згадки @user
    text = re.sub(r"http\S+", " ", text)                      # прибрати посилання
    text = re.sub(r"<.*?>", " ", text)                        # прибрати HTML-теги
    text = re.sub(r"[^a-z\s]", " ", text)                     # залишити лише букви (прибрати цифри, спецсимволи)
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)  # прибрати пунктуацію
    text = re.sub(r"\s+", " ", text)                          # замінити багато пробілів на один
    return text

In [5]:
cleaned_texts = [clean_text(ex["text"]) for ex in ds["train"]]
tokens = [word_tokenize(t) for t in cleaned_texts]

In [6]:
counter = Counter()
for tok_list in tokens:
    counter.update(tok_list)

In [7]:
# кількість унікальних слів
vocab_size = len(counter)

print(f"Кількість унікальних слів у train-наборі: {vocab_size:,}")

Кількість унікальних слів у train-наборі: 73,206


In [8]:
 # обмежимо словник
MAX_VOCAB = 20_000

specials = ["[PAD]", "[UNK]", "[BOS]", "[EOS]"]
most_common = counter.most_common(MAX_VOCAB - len(specials))
itos = specials + [w for w, _ in most_common]
stoi = {w: i for i, w in enumerate(itos)}

PAD_IDX = stoi["[PAD]"]
UNK_IDX = stoi["[UNK]"]
BOS_IDX = stoi["[BOS]"]
EOS_IDX = stoi["[EOS]"]

print("Розмір словника:", len(stoi))
print("PAD:", PAD_IDX, "UNK:", UNK_IDX, "BOS:", BOS_IDX, "EOS:", EOS_IDX)


Розмір словника: 20000
PAD: 0 UNK: 1 BOS: 2 EOS: 3


In [9]:
MAX_LEN = 100   # довжина з BOS/EOS

def encode_with_specials(tok_list):
    # звичайні слова → індекси
    ids = [stoi.get(t, UNK_IDX) for t in tok_list]
    # ріжемо, щоб залишилось місце для BOS і EOS
    ids = ids[:MAX_LEN - 2]
    # додаємо спеціальні токени
    return [BOS_IDX] + ids + [EOS_IDX]

encoded_texts = [encode_with_specials(tok_list) for tok_list in tokens]
print(encoded_texts[0][:20])


[2, 12, 1595, 12, 241, 1974, 4047, 40, 62, 368, 1099, 88, 7, 33, 4, 6976, 14, 3358, 10, 56]


In [10]:
#Padding
def pad_sequence(seq):
    seq = seq[:MAX_LEN]
    seq = seq + [PAD_IDX] * max(0, MAX_LEN - len(seq))
    return torch.tensor(seq, dtype=torch.long)

In [11]:
#input/таргет (зсув на 1)
def make_lm_pair(seq):
    # seq вже містить [BOS ... EOS]
    inp = seq[:-1]   # [BOS ... останнє слово]
    tgt = seq[1:]    # [... EOS]
    return inp, tgt

pairs = [make_lm_pair(seq) for seq in encoded_texts]

X_in  = torch.stack([pad_sequence(inp) for inp, _ in pairs])
Y_tgt = torch.stack([pad_sequence(tgt) for _, tgt in pairs])

print(X_in.shape, Y_tgt.shape)   # [N, MAX_LEN]
print(X_in[0][:10])
print(Y_tgt[0][:10])

torch.Size([25000, 100]) torch.Size([25000, 100])
tensor([   2,   12, 1595,   12,  241, 1974, 4047,   40,   62,  368])
tensor([  12, 1595,   12,  241, 1974, 4047,   40,   62,  368, 1099])


In [12]:
#TensorDataset і DataLoader
dataset = TensorDataset(X_in, Y_tgt) # !!!

# розділяємо на train/val
VAL_FRAC = 0.2
val_sz = int(len(dataset) * VAL_FRAC)
train_sz = len(dataset) - val_sz
train_ds, val_ds = random_split(dataset, [train_sz, val_sz],
                                generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=128)


In [13]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        # pe: [max_len, d_model]
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)          # [max_len, 1]
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # [1, max_len, d_model] – для додавання до batch
        self.register_buffer("pe", pe)

    def forward(self, x):
        # x: [B, T, d_model]
        x = x + self.pe[:, : x.size(1)]
        return x

In [14]:
class TransformerDecoderLM(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        pad_idx,
        n_heads=4,
        n_layers=2,
        dim_feedforward=256,
        dropout=0.1,
        max_len=5000,
    ):
        super().__init__()
        self.pad_idx = pad_idx
        self.emb_dim = emb_dim

        # Слово -> вектор
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            padding_idx=pad_idx,
        )

        # Позиційне кодування
        self.pos_encoding = PositionalEncoding(d_model=emb_dim, max_len=max_len)

        # Один шар декодера трансформера
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=emb_dim,
            nhead=n_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
        )

        # Стек із кількох шарів
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=n_layers)

        # Проекція у простір словника
        self.fc_out = nn.Linear(emb_dim, vocab_size)

    def _generate_square_subsequent_mask(self, T, device):
        # каузальна маска: кожна позиція "бачить" тільки попередні слова
        mask = torch.triu(torch.ones(T, T, device=device), diagonal=1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask

    def forward(self, x):
        # x: [B, T]
        emb = self.embedding(x)           # [B, T, D]
        emb = self.pos_encoding(emb)      # [B, T, D]

        B, T, D = emb.shape
        tgt_mask = self._generate_square_subsequent_mask(T, x.device)

        out = self.decoder(
            tgt=emb,
            memory=emb,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=(x == self.pad_idx),
            memory_key_padding_mask=(x == self.pad_idx),
        )                                 # [B, T, D]

        logits = self.fc_out(out)         # [B, T, V]
        return logits


In [15]:
model = TransformerDecoderLM(
    vocab_size=len(stoi),
    emb_dim=128,
    pad_idx=PAD_IDX,
    n_heads=4,
    n_layers=2,
    dim_feedforward=256,
    dropout=0.1,
    max_len=MAX_LEN,
).to(device)

In [16]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [17]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0

    for x, y in loader:
        x, y = x.to(device), y.to(device)      # x,y: [B, T]

        optimizer.zero_grad()
        logits = model(x)                      # [B, T, V]

        B, T, V = logits.shape
        loss = criterion(
            logits.view(B*T, V),
            y.view(B*T)
        )
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * B

    return running_loss / len(loader.dataset)

In [18]:
def evaluate_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)

            B, T, V = logits.shape
            loss = criterion(
                logits.view(B*T, V),
                y.view(B*T)
            )

            running_loss += loss.item() * B

    return running_loss / len(loader.dataset)


In [19]:
EPOCHS = 30

for ep in range(1, EPOCHS+1):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate_one_epoch(model, val_loader, criterion, device)
    print(f"Epoch {ep}/{EPOCHS} - loss: {train_loss:.4f} - val_loss: {val_loss:.4f}")


/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:6041: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch 1/30 - loss: 6.6926 - val_loss: 5.8310
Epoch 2/30 - loss: 5.1204 - val_loss: 4.0801
Epoch 3/30 - loss: 3.3228 - val_loss: 2.2321
Epoch 4/30 - loss: 1.8240 - val_loss: 1.1870
Epoch 5/30 - loss: 1.0335 - val_loss: 0.7400
Epoch 6/30 - loss: 0.6355 - val_loss: 0.5175
Epoch 7/30 - loss: 0.4188 - val_loss: 0.3860
Epoch 8/30 - loss: 0.2868 - val_loss: 0.3088
Epoch 9/30 - loss: 0.2032 - val_loss: 0.2638
Epoch 10/30 - loss: 0.1491 - val_loss: 0.2285
Epoch 11/30 - loss: 0.1098 - val_loss: 0.2099
Epoch 12/30 - loss: 0.0834 - val_loss: 0.1904
Epoch 13/30 - loss: 0.0642 - val_loss: 0.1817
Epoch 14/30 - loss: 0.0503 - val_loss: 0.1681
Epoch 15/30 - loss: 0.0405 - val_loss: 0.1646
Epoch 16/30 - loss: 0.0329 - val_loss: 0.1562
Epoch 17/30 - loss: 0.0272 - val_loss: 0.1496
Epoch 18/30 - loss: 0.0231 - val_loss: 0.1461
Epoch 19/30 - loss: 0.0196 - val_loss: 0.1441
Epoch 20/30 - loss: 0.0171 - val_loss: 0.1390
Epoch 21/30 - loss: 0.0152 - val_loss: 0.1387
Epoch 22/30 - loss: 0.0137 - val_loss: 0.13

In [21]:
itos = {i: w for w, i in stoi.items()}

def generate(model, start_text, max_new_tokens=20):
    model.eval()
    with torch.no_grad():
        # токенізуємо й чистимо
        tokens = word_tokenize(clean_text(start_text))
        ids = [BOS_IDX] + [stoi.get(t, UNK_IDX) for t in tokens]
        ids = ids[:MAX_LEN-1]  # щоб був запас для EOS

        seq = torch.tensor([ids], dtype=torch.long, device=device)

        for _ in range(max_new_tokens):
            logits = model(seq)              # [1, T, V]
            next_logits = logits[0, -1]      # остання позиція

            temperature = 2.0
            probs = torch.softmax(next_logits / temperature, dim=-1)


            next_idx = torch.multinomial(probs, num_samples=1).item()

            seq = torch.cat(
                [seq, torch.tensor([[next_idx]], device=device)],
                dim=1
            )

            if next_idx == EOS_IDX:
                break

        # перетворюємо назад у слова, ігноруємо PAD/BOS/EOS
        ids = seq[0].cpu().tolist()
        words = []
        for idx in ids:
            if idx in (PAD_IDX, BOS_IDX, EOS_IDX):
                continue
            words.append(itos.get(idx, "<UNK>"))
        return " ".join(words)


print(generate(model, "this movie was", max_new_tokens=10))


this movie was was slayer wore wore mraovich daisy was was was kissing
